In [75]:
import pandas as pd
import csv
import json
import numpy as np

In [76]:
data = pd.read_csv('cardekho.csv')

In [77]:
# replace ' ' with NaN
data.replace(' ', np.nan, inplace=True)
data['max_power'] = data['max_power'].astype(float)

In [78]:
# group by car name and fill null values in engine, power, seats with mean of that car
data['engine'] = data.groupby('name')['engine'].transform(lambda x: x.fillna(x.mean()))
data['max_power'] = data.groupby('name')['max_power'].transform(lambda x: x.fillna(x.mean()))
data['seats'] = data.groupby('name')['seats'].transform(lambda x: x.fillna(x.mean()))

In [79]:
data.dropna(inplace=True)

In [80]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7906 entries, 0 to 8127
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                7906 non-null   object 
 1   year                7906 non-null   int64  
 2   selling_price       7906 non-null   int64  
 3   km_driven           7906 non-null   int64  
 4   fuel                7906 non-null   object 
 5   seller_type         7906 non-null   object 
 6   transmission        7906 non-null   object 
 7   owner               7906 non-null   object 
 8   mileage(km/ltr/kg)  7906 non-null   float64
 9   engine              7906 non-null   float64
 10  max_power           7906 non-null   float64
 11  seats               7906 non-null   float64
dtypes: float64(4), int64(3), object(5)
memory usage: 803.0+ KB


In [81]:
data['manufacturer'] = data['name'].apply(lambda x: x.split()[0])

In [82]:
data['owner'] = data['owner'].apply(lambda x: x.split()[0])

In [83]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage(km/ltr/kg),engine,max_power,seats,manufacturer
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First,23.40,1248.0,74.00,5.0,Maruti
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second,21.14,1498.0,103.52,5.0,Skoda
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third,17.70,1497.0,78.00,5.0,Honda
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First,23.00,1396.0,90.00,5.0,Hyundai
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First,16.10,1298.0,88.20,5.0,Maruti


In [84]:
data.drop(['name', 'selling_price'], axis=1, inplace=True)

In [85]:
data.head()

,year,km_driven,fuel,seller_type,transmission,owner,mileage(km/ltr/kg),engine,max_power,seats,manufacturer
0,2014,145500,Diesel,Individual,Manual,First,23.40,1248.0,74.00,5.0,Maruti
1,2014,120000,Diesel,Individual,Manual,Second,21.14,1498.0,103.52,5.0,Skoda
2,2006,140000,Petrol,Individual,Manual,Third,17.70,1497.0,78.00,5.0,Honda
3,2010,127000,Diesel,Individual,Manual,First,23.00,1396.0,90.00,5.0,Hyundai
4,2007,120000,Petrol,Individual,Manual,First,16.10,1298.0,88.20,5.0,Maruti


In [86]:
categorical = ['fuel', 'seller_type', 'transmission', 'owner', 'manufacturer']  # categorical columns
ranged = ['year', 'km_driven', 'mileage(km/ltr/kg)', 'engine', 'max_power', 'seats']  # range columns

In [90]:
# remove manufacturer with less than 10 cars
data = data.groupby('manufacturer').filter(lambda x: len(x) > 10)

In [93]:
# create options.json file
# for categorical columns, get unique values
# for range columns, get min and max values
options = {}
for col in categorical:
    options[col] = list(data[col].unique())
for col in ranged:
    options[col] = {'min': float(data[col].min()), 'max': float(data[col].max())}

In [94]:
# write options_fixed to options.json
with open('options.json', 'w') as f:
    json.dump(options, f)